In [2]:
from DNASkittleUtils.Contigs import *
contigs = read_contigs(r"D:\josiah\Documents\Research\Thesis - Genome Symmetry\Green_Ash_Annotation_Transfer\fraxinus_pennsylvanica_20Feb2018_lPmM4_psuedochromosomes__gold_standard.fasta")

In [3]:
other_contigs = read_contigs(r"D:\josiah\Documents\Research\Thesis - Genome Symmetry\Green_Ash_Annotation_Transfer\fraxinus_pennsylvanica_20Feb2018_lPmM4_psuedochromosomes.fasta")

In [4]:
len(contigs), len(other_contigs)

(23, 23)

In [5]:
paired = zip(contigs, other_contigs)
[len(a.seq) == len(b.seq) for a,b in paired]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [6]:
source = read_contigs("fraxinus_pennsylvanica_20Feb2018_lPmM4.fasta")

FileNotFoundError: [Errno 2] No such file or directory: 'fraxinus_pennsylvanica_20Feb2018_lPmM4.fasta'